In [1]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from itertools import product
import numpy as np
import pandas as pd
import warnings
import matplotlib as plt
import seaborn as sn

warnings.filterwarnings("ignore")

## Import Data

In [2]:
df_oferta = pd.read_csv('../data/datos_Oferta_Renovables_EU27_GWh.csv', index_col=0, parse_dates=True)
# establecer frecuencia diaria
df_oferta = df_oferta.asfreq('D')

# tomar la serie de datos acumuados en EU27
serie_O = df_oferta['EU27'].copy()
serie_O.index

DatetimeIndex(['1980-01-01', '1980-01-02', '1980-01-03', '1980-01-04',
               '1980-01-05', '1980-01-06', '1980-01-07', '1980-01-08',
               '1980-01-09', '1980-01-10',
               ...
               '2024-12-22', '2024-12-23', '2024-12-24', '2024-12-25',
               '2024-12-26', '2024-12-27', '2024-12-28', '2024-12-29',
               '2024-12-30', '2024-12-31'],
              dtype='datetime64[ns]', name='Date', length=16437, freq='D')

In [3]:
serie_O.to_csv('data_serie_oferta_EU27_GWh.csv', index=True)

## Modelos ARIMA(p,0,q)

In [4]:
def obtener_df_modelos_ARIMA(serie_temp, p_values:list, q_values:list, d, METRICA_EVAL):
    '''
    Función para entrenar y evaluar modelos ARIMA en una serie temporal.
    Parametros:
    - serie_temp: Serie temporal a evaluar (pd.Series).
    - p_values: Lista de valores para el parámetro p (int).
    - q_values: Lista de valores para el parámetro q (int).
    - d: Diferenciación (int).
    - METRICA_EVAL: Métrica de evaluación (str) para ordenar los resultados ('AIC', 'BIC', 'RMSE', 'MAPE', 'R2').
    '''
    
    dict_metricas_eval = {
        'AIC': True,  # AIC menor es mejor (ascending=True)
        'BIC': True,  # BIC menor es mejor
        'RMSE': True,  # RMSE menor es mejor
        'MAPE': True,  # MAPE menor es mejor
        'R2': False  # R2 mayor es mejor (ascending=False)
    }

    # Train-test split
    train_size = int(len(serie_temp) * 0.8)
    train, test = serie_temp[:train_size], serie_temp[train_size:]

    # Crear combinaciones de parámetros
    param_combinations = list(product(p_values, q_values))
    P, D, Q, m = 0, 0, 0, 0  # no estacionalidad 

    # Resultados
    results = []

    # Loop por combinaciones
    for i, (p, q) in enumerate(param_combinations):
        try:
            print()
            print(f'Entrenando modelo ARIMA({p},{d},{q})...')
            model = SARIMAX(train,
                            order=(p, d, q),
                            seasonal_order=(P, D, Q, m),
                            enforce_stationarity=False,
                            enforce_invertibility=False)
            fitted_model = model.fit(disp=False)  

            # Forecast
            print('Prediciendo...')
            pred = fitted_model.forecast(steps=len(test))

            # Evaluación
            print('Evaluando...')
            aic = fitted_model.aic
            bic = fitted_model.bic
            rmse = np.sqrt(mean_squared_error(test, pred))
            mape = mean_absolute_percentage_error(test, pred)
            r2 = r2_score(test, pred)

            results.append({
                'p': p, 'q': q, #'P': P, 'Q': Q,
                'AIC': aic,
                'BIC': bic,
                'RMSE': rmse,
                'MAPE': mape,
                'R2': r2,
                'params': fitted_model.params
            })

            print(f'Modelo {i+1}/{len(param_combinations)} listo.')

        except Exception as e:
            print(f'Error en modelo ARIMA({p},{d},{q}): {e} de tipo {type(e).__name__}')
            continue

    # Convertir a DataFrame y ordenar por AIC (cuanto menor, mejor)
    df_arima = pd.DataFrame(results).sort_values(f'{METRICA_EVAL}', ascending=dict_metricas_eval[METRICA_EVAL])
    return df_arima

### ARIMA parámetros significativos
parámetros teóricamente más significativos:
- ```d=0``` :  serie_O estacionaria según test ADF y KPSS
- ```p``` : por el PACF vemos que los lags más significativos son ```[1,2,3]```
- ```q``` : por el ACF vemos que los lags más significativos son ```[1,2,3]```


#### serie parcial 2015-2024

In [5]:
serie_O_ej_01 = serie_O['2015':'2024']

df_arima_01 = obtener_df_modelos_ARIMA(serie_O_ej_01, 
                                       p_values=[1,2,3], 
                                       q_values=[1,2,3], 
                                       d=0, 
                                       METRICA_EVAL='R2')
df_arima_01


Entrenando modelo ARIMA(1,0,1)...
Prediciendo...
Evaluando...
Modelo 1/9 listo.

Entrenando modelo ARIMA(1,0,2)...
Prediciendo...
Evaluando...
Modelo 2/9 listo.

Entrenando modelo ARIMA(1,0,3)...
Prediciendo...
Evaluando...
Modelo 3/9 listo.

Entrenando modelo ARIMA(2,0,1)...
Prediciendo...
Evaluando...
Modelo 4/9 listo.

Entrenando modelo ARIMA(2,0,2)...
Prediciendo...
Evaluando...
Modelo 5/9 listo.

Entrenando modelo ARIMA(2,0,3)...
Prediciendo...
Evaluando...
Modelo 6/9 listo.

Entrenando modelo ARIMA(3,0,1)...
Prediciendo...
Evaluando...
Modelo 7/9 listo.

Entrenando modelo ARIMA(3,0,2)...
Prediciendo...
Evaluando...
Modelo 8/9 listo.

Entrenando modelo ARIMA(3,0,3)...
Prediciendo...
Evaluando...
Modelo 9/9 listo.


,p,q,AIC,BIC,RMSE,MAPE,R2,params
2,1,3,41757.174318,41787.067587,496.377152,0.210414,-0.006244,ar.L1 0.999698 ma.L1 -0.165040 ...
5,2,3,41702.196015,41738.067937,503.527610,0.201678,-0.035443,ar.L1 1.411059 ar.L2 -0.411178 ...
6,3,1,41736.399832,41766.294814,504.826941,0.201290,-0.040794,ar.L1 1.802360 ar.L2 -1.00264...
4,2,2,41714.280965,41744.175947,506.799938,0.200525,-0.048945,ar.L1 1.508965 ar.L2 -0.50906...
8,3,3,41699.262322,41741.112898,507.615746,0.200348,-0.052325,ar.L1 1.120617 ar.L2 0.076951 ...
7,3,2,41789.282998,41825.156977,547.483058,0.198995,-0.224112,ar.L1 1.457361 ar.L2 -0.26159...
1,1,2,41941.955729,41965.871714,945.026752,0.386416,-2.647270,ar.L1 0.997895 ma.L1 -0.18494...
0,1,1,42232.379406,42250.317422,1847.378280,0.883932,-12.937710,ar.L1 0.984379 ma.L1 0.02106...
3,2,1,42203.198834,42227.116190,1855.667417,0.889477,-13.063067,ar.L1 0.409121 ar.L2 0.56503...


In [6]:
# ordenando por AIC
df_arima_01.sort_values('AIC', ascending=True)

,p,q,AIC,BIC,RMSE,MAPE,R2,params
8,3,3,41699.262322,41741.112898,507.615746,0.200348,-0.052325,ar.L1 1.120617 ar.L2 0.076951 ...
5,2,3,41702.196015,41738.067937,503.527610,0.201678,-0.035443,ar.L1 1.411059 ar.L2 -0.411178 ...
4,2,2,41714.280965,41744.175947,506.799938,0.200525,-0.048945,ar.L1 1.508965 ar.L2 -0.50906...
6,3,1,41736.399832,41766.294814,504.826941,0.201290,-0.040794,ar.L1 1.802360 ar.L2 -1.00264...
2,1,3,41757.174318,41787.067587,496.377152,0.210414,-0.006244,ar.L1 0.999698 ma.L1 -0.165040 ...
7,3,2,41789.282998,41825.156977,547.483058,0.198995,-0.224112,ar.L1 1.457361 ar.L2 -0.26159...
1,1,2,41941.955729,41965.871714,945.026752,0.386416,-2.647270,ar.L1 0.997895 ma.L1 -0.18494...
3,2,1,42203.198834,42227.116190,1855.667417,0.889477,-13.063067,ar.L1 0.409121 ar.L2 0.56503...
0,1,1,42232.379406,42250.317422,1847.378280,0.883932,-12.937710,ar.L1 0.984379 ma.L1 0.02106...


#### datos completos 1980-2024


In [6]:
# parámetros significativos
df_arima_teoria = obtener_df_modelos_ARIMA(serie_O, 
                                        p_values=[1,2,3], 
                                        q_values=[1,2,3], 
                                        d=0, 
                                        METRICA_EVAL='R2')
df_arima_teoria


Entrenando modelo ARIMA(1,0,1)...
Prediciendo...
Evaluando...
Modelo 1/9 listo.

Entrenando modelo ARIMA(1,0,2)...
Prediciendo...
Evaluando...
Modelo 2/9 listo.

Entrenando modelo ARIMA(1,0,3)...
Prediciendo...
Evaluando...
Modelo 3/9 listo.

Entrenando modelo ARIMA(2,0,1)...
Prediciendo...
Evaluando...
Modelo 4/9 listo.

Entrenando modelo ARIMA(2,0,2)...
Prediciendo...
Evaluando...
Modelo 5/9 listo.

Entrenando modelo ARIMA(2,0,3)...
Prediciendo...
Evaluando...
Modelo 6/9 listo.

Entrenando modelo ARIMA(3,0,1)...
Prediciendo...
Evaluando...
Modelo 7/9 listo.

Entrenando modelo ARIMA(3,0,2)...
Prediciendo...
Evaluando...
Modelo 8/9 listo.

Entrenando modelo ARIMA(3,0,3)...
Prediciendo...
Evaluando...
Modelo 9/9 listo.


,p,q,AIC,BIC,RMSE,MAPE,R2
7,3,2,188756.443206,188801.346443,606.326372,0.218216,-0.604054
8,3,3,188716.911076,188769.297653,612.418354,0.220630,-0.636449
5,2,3,188720.846905,188765.749686,614.617859,0.221398,-0.648224
4,2,2,188767.430035,188804.849399,639.350328,0.231507,-0.783544
6,3,1,188878.933381,188916.352745,643.588527,0.233247,-0.807268
2,1,3,188954.809235,188992.228218,897.641435,0.358100,-2.515699
1,1,2,189712.929685,189742.865176,1750.672880,0.853603,-12.372603
3,2,1,191134.043722,191163.979517,1926.955690,0.983892,-15.201283
0,1,1,191170.562870,191193.014716,1928.381747,0.984771,-15.225272


In [10]:
df_arima_teoria.to_csv('resultados_modelos/ARIMA_parametros_significativos_R2.csv', index=False)

In [13]:
df_arima_teoria_AIC = df_arima_teoria.sort_values('AIC', ascending=True)
df_arima_teoria_AIC

,p,q,AIC,BIC,RMSE,MAPE,R2
8,3,3,188716.911076,188769.297653,612.418354,0.220630,-0.636449
5,2,3,188720.846905,188765.749686,614.617859,0.221398,-0.648224
7,3,2,188756.443206,188801.346443,606.326372,0.218216,-0.604054
4,2,2,188767.430035,188804.849399,639.350328,0.231507,-0.783544
6,3,1,188878.933381,188916.352745,643.588527,0.233247,-0.807268
2,1,3,188954.809235,188992.228218,897.641435,0.358100,-2.515699
1,1,2,189712.929685,189742.865176,1750.672880,0.853603,-12.372603
3,2,1,191134.043722,191163.979517,1926.955690,0.983892,-15.201283
0,1,1,191170.562870,191193.014716,1928.381747,0.984771,-15.225272


In [14]:
df_arima_teoria_AIC.to_csv('resultados_modelos/ARIMA_parametros_significativos_AIC.csv', index=False)

## Invertibilidad modelos

In [ ]:
# Train-test split
train_size = int(len(serie_O) * 0.8)
train, test = serie_O[:train_size], serie_O[train_size:]

# modelo ARIMA(3,0,2)
model_1 = SARIMAX(train, order=(3, 0, 2), seasonal_order=(0, 0, 0, 0))
fitted_model = model_1.fit()

# Extraer los coeficientes AR del modelo ajustado
ar_params = fitted_model.arparams  # Parámetros AR
print("Coeficientes AR:", ar_params)

# Comprobar la invertibilidad
from statsmodels.tsa.arima_process import ArmaProcess
ar_roots = ArmaProcess(ar=[1] + list(-ar_params)).arroots
print("Raíces del polinomio AR:", ar_roots)

if all(abs(root) > 1 for root in ar_roots):
    print("El modelo es invertible.")
else:
    print("El modelo NO es invertible.")

Coeficientes AR: [ 1.4917121  -0.48161501 -0.01022035]
Raíces del polinomio AR: [-50.07680686   1.00024764   1.95339584]
El modelo es invertible.
